In [1]:
import requests
import datetime
import sqlite3
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
db_path = './fantasy_nfl.db'
table_name = 'player_proj'
html_string = requests.get('https://www.numberfire.com/nfl/fantasy/fantasy-football-ppr-projections').text

In [3]:
soup = BeautifulSoup(html_string, 'lxml') # Parse the HTML as a string
table = soup.findAll('table',attrs={'class':'projection-table'})

In [4]:
## Draw from first table to get player details
rows = table[0].findAll('tr')
row_clipped = rows[2:]
full_name = [row.find('span',attrs={'class':'full'}).text for row in row_clipped]
index = [int(row.attrs['data-row-index']) for row in row_clipped]
pos_team = [row.find('td',{'class':'player'}).text.split('\n')[-2].strip(' ') for row in row_clipped]
pos = [v.split(', ')[0].strip("(") for v in pos_team]
team = [v.split(', ')[1].strip(")") for v in pos_team]

player_details = pd.DataFrame.from_dict({
    'index':index,
    'full_name':full_name,
    'pos':pos,
    'team':team}).set_index('index')

In [5]:
## Draw from second table to get projections
rows = table[1].findAll('tr')
headers = [v.attrs['title'] for v in rows[1].findAll('th')]

count = 1
val_dict = {}
for row in rows[2:]:
    val_dict[count] = [v.text.replace(' ','').strip('\n') for v in row.findAll('td')]
    count += 1

proj = pd.DataFrame.from_dict(val_dict, orient='index',columns=headers)

In [6]:
## Join tables
final = player_details.join(proj)
final['pull_datetime'] = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')

In [7]:
con = sqlite3.connect(db_path)
final.to_sql(name=table_name, con=con, if_exists='append', index=False)
con.close()

/Users/ryand/.pyenv/versions/3.6.1/envs/jupyter3/lib/python3.6/site-packages/pandas/core/generic.py:2663: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [8]:
con = sqlite3.connect(db_path)
fp_df = pd.read_sql(con=con, sql="SELECT * FROM '{}' WHERE pull_datetime=(SELECT MAX(pull_datetime) FROM '{}')".format(table_name,table_name))
con.close()

In [11]:
fp_df['full_name'] fp_df['']

,full_name,pos,team,Standard Fantasy Points,Opponent Team,Opponent Team Defensive Rank,Player Overall Rank,Player Position Rank,Passing Yards,Passing Touchdowns,...,FanDuel Fantasy Points,FanDuel Cost,FanDuel Value,DraftKings Fantasy Points,DraftKings Cost,DraftKings Value,Yahoo Fantasy Points,Yahoo Cost,Yahoo Value,pull_datetime
0,Patrick Mahomes,QB,KC,23.7,BUF,#24,#1,#1,289.89,2.58,...,24.37,N/A,N/A,24.56,$7800,3.15,24.37,$39,0.62,2020-10-14 13:12
1,Kyler Murray,QB,ARI,22.57,DAL,#30,#2,#2,267.02,2.06,...,23.35,N/A,N/A,23.5,$7300,3.22,23.35,$38,0.61,2020-10-14 13:12
2,Lamar Jackson,QB,BAL,22.43,PHI,#29,#3,#3,238.60,1.90,...,23.05,$9000,2.56,23.32,$7700,3.03,23.05,$35,0.66,2020-10-14 13:12
3,Josh Allen,QB,BUF,21.91,KC,#9,#4,#4,258.98,1.84,...,22.7,N/A,N/A,23.04,$7900,2.92,22.7,$41,0.55,2020-10-14 13:12
4,Derrick Henry,RB,TEN,21.77,HOU,#27,#5,#1,0.00,0.00,...,20.56,$9000,2.28,24.95,$7300,3.42,20.56,$39,0.53,2020-10-14 13:12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444,Jarrett Stidham,QB,NE,0,DEN,#10,#390,#31,0.00,0.00,...,0,$6900,N/A,0,$5400,N/A,0,$20,N/A,2020-10-14 13:12
445,Steven Sims,WR,WSH,0,NYG,#19,#390,#165,0.00,0.00,...,0,$4600,N/A,0,$3400,N/A,0,$10,N/A,2020-10-14 13:12
446,Reggie Bonnafon,RB,CAR,0,CHI,#13,#390,#112,0.00,0.00,...,0,$4700,N/A,0,N/A,N/A,0,N/A,N/A,2020-10-14 13:12
447,Ryquell Armstead,RB,JAC,0,DET,#22,#390,#112,0.00,0.00,...,0,$4500,N/A,0,$4000,N/A,0,$10,N/A,2020-10-14 13:12
